<h1 style="text-align:center"> <b>Image Classification using Transfer Learning in PyTorch <b/><h1>

Face masks have become a common public sight in the last few months. The Centers for Disease Control (CDC) recently advised the use of simple cloth face coverings to slow the spread of the virus and help people who may have the virus and do not know it from transmitting it to others. Wearing masks is broadly recognised as critical to reducing community transmission and limiting touching of the face.

In a time of concerns about slowing the transmission of COVID-19, increased surveillance combined with AI solutions can improve monitoring and reduce the human effort needed to limit the spread of this disease. The objective of this challenge is to create an image classification machine learning model to accurately predict the likelihood that an image contains a person wearing a face mask, or not. The total dataset contains 1,800+ images of people either wearing masks or not.

Our machine learning solution will help policymakers, law enforcement, hospitals, and even commercial businesses ensure that masks are being worn appropriately in public. These solutions can help in the battle to reduce community transmission of COVID-19

<h2>Import librarys and modules</h2>

In [1]:
import pandas as pd
import numpy as np
import torchvision
import os
from torchvision import transforms, utils
from PIL import Image
from torchvision.transforms import ToTensor
from torch.autograd import Variable
import torch
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
import PIL 
from torch import nn
import time
import random
from matplotlib import gridspec

In [2]:
print(torchvision.__version__)
print(torch.__version__)
print(PIL.__version__)

0.15.1+cu117
2.0.0+cu117
9.4.0


<h2>Set a seed for the random initialization of weights provided by the nn module</h2>

In [4]:
manualSeed = 1
random_seed= 42
np.random.seed(manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
# if you are suing GPU
torch.cuda.manual_seed(manualSeed)
torch.cuda.manual_seed_all(manualSeed)


torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

<h2>Download Data</h2>

In [ ]:
!wget -O images.zip --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1vtO5_FB--urbF09XYoxbSAEQW-yqwoi6' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1vtO5_FB--urbF09XYoxbSAEQW-yqwoi6" && rm -rf /tmp/cookies.txt

In [ ]:
!wget -O train.csv --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1XQ-RfUG1C_6LUZdRUe9nAU77I_AZFH-F' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1XQ-RfUG1C_6LUZdRUe9nAU77I_AZFH-F" && rm -rf /tmp/cookies.txt

In [ ]:
!unzip '/content/images.zip'

In [5]:
train = pd.read_csv('./train_labels.csv')

train_images_list = train['image'].tolist()

images_list = os.listdir('./images')

test_images_list = [fn for fn in images_list if fn not in train_images_list]


In [6]:
image_transform=transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

<h2>Create Test images DateFrame</h2>

In [7]:
sub=pd.DataFrame({
    'image':test_images_list,
     'target':0
})


<h2>Transform both train and validation Images</h2>

In [8]:
train_img=[]
ref = pd.read_csv('./train_labels.csv')

for i in range(0,ref.shape[0]) :
   img_path="./images/"+str(ref.iloc[i,:].image)
   image = Image.open(img_path).convert('RGB')
   image=image_transform(image)
   train_img.append((image,ref.iloc[i,:].target))

<h2>Download Pre-trained ResNet50 model</h2>

In [19]:
resnet50 = torchvision.models.resnet50(pretrained=True)


<h2> Don't Freeze model parameters</h2>

In [20]:

for param in resnet50.parameters():
    param.requires_grad = True

<h4>Replace the final layer of the ResNet50 model by a small set of Sequential layers. The inputs to the last fully connected layer of ResNet50 is fed to a Linear layer which has 128 outputs, which are then fed into ReLU and Dropout layers. It is then followed by a 128×2 Linear Layer which has 2 outputs corresponding to the 2 classes (mask , without mask).</h4>

In [21]:

fc_inputs = resnet50.fc.in_features
 
resnet50.fc = nn.Sequential(
    nn.Linear(fc_inputs, 128),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(128,2),
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)


In [22]:
model = resnet50.to('cpu') 


<h2>Define Optimizer and Loss Function</h2>

In [33]:
#we have reduced the learning rate to avoid overfitting
loss_func = nn.NLLLoss()
optimizer =torch.optim.Adam(model.parameters(), lr=0.000099)


<h2>Creating data indices for training and validation splits</h2>

In [34]:

batch_size = 32
validation_split = .0
shuffle_dataset = True
random_seed= 42
dataset_size = len(train_img)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(train_img, batch_size=batch_size, 
                                           sampler=train_sampler)

validation_loader = torch.utils.data.DataLoader(train_img, batch_size=batch_size,
                                                sampler=valid_sampler)

In [35]:
device = torch.device("cpu")

<H2>Train<h2>

In [36]:
epochs=5                           

for epoch in range(epochs):

    epoch_start = time.time()
    print("Epoch: {}/{}".format(epoch+1, epochs))
    # Set to training mode
    model.train()
     
    # Loss and Accuracy within the epoch
    train_loss = 0.0
    train_acc = 0.0
     
    valid_loss = 0.0
    valid_acc = 0.0
 
    for i, (inputs, labels) in enumerate(train_loader):
 
        inputs = inputs.to(device)
        labels = labels.to(device)
         
        # Clean existing gradients
        optimizer.zero_grad()
         
        # Forward pass - compute outputs on input data using the model
        outputs = model(inputs)
         
        # Compute loss
        #print(outputs)
        loss = loss_func(outputs, labels)
         
        # Backpropagate the gradients
        loss.backward()
         
        # Update the parameters
        optimizer.step()
         
        # Compute the total loss for the batch and add it to train_loss
        train_loss += loss.item() * inputs.size(0)
         
        # Compute the accuracy
        ret, predictions = torch.max(outputs.data, 1)
        correct_counts = predictions.eq(labels.data.view_as(predictions))
         
        # Convert correct_counts to float and then compute the mean
        acc = torch.mean(correct_counts.type(torch.FloatTensor))
         
        # Compute total accuracy in the whole batch and add to train_acc
        train_acc += acc.item() * inputs.size(0)
         
        print("Batch number: {:03d}, Training: Loss: {:.8f}, Accuracy: {:.8f}".format(i, loss.item(), acc.item()))
    

Epoch: 1/5
Batch number: 000, Training: Loss: 0.71856731, Accuracy: 0.50000000
Batch number: 001, Training: Loss: 0.67792588, Accuracy: 0.50000000
Batch number: 002, Training: Loss: 0.62004030, Accuracy: 0.75000000
Batch number: 003, Training: Loss: 0.60241359, Accuracy: 0.78125000
Batch number: 004, Training: Loss: 0.56288576, Accuracy: 0.81250000
Batch number: 005, Training: Loss: 0.52642846, Accuracy: 0.81250000
Batch number: 006, Training: Loss: 0.50113016, Accuracy: 0.84375000
Batch number: 007, Training: Loss: 0.43169802, Accuracy: 0.90625000
Batch number: 008, Training: Loss: 0.43047649, Accuracy: 0.90625000
Batch number: 009, Training: Loss: 0.34210378, Accuracy: 0.93750000
Batch number: 010, Training: Loss: 0.30509290, Accuracy: 0.93750000
Batch number: 011, Training: Loss: 0.34529108, Accuracy: 0.81250000
Batch number: 012, Training: Loss: 0.25597644, Accuracy: 0.96875000
Batch number: 013, Training: Loss: 0.18932751, Accuracy: 1.00000000
Batch number: 014, Training: Loss: 0.

Batch number: 038, Training: Loss: 0.09316438, Accuracy: 0.96875000
Batch number: 039, Training: Loss: 0.00923903, Accuracy: 1.00000000
Batch number: 040, Training: Loss: 0.00795466, Accuracy: 1.00000000
Epoch: 4/5
Batch number: 000, Training: Loss: 0.00793010, Accuracy: 1.00000000
Batch number: 001, Training: Loss: 0.00976410, Accuracy: 1.00000000
Batch number: 002, Training: Loss: 0.00421497, Accuracy: 1.00000000
Batch number: 003, Training: Loss: 0.00837757, Accuracy: 1.00000000
Batch number: 004, Training: Loss: 0.00258501, Accuracy: 1.00000000
Batch number: 005, Training: Loss: 0.00566318, Accuracy: 1.00000000
Batch number: 006, Training: Loss: 0.01958037, Accuracy: 1.00000000
Batch number: 007, Training: Loss: 0.01570089, Accuracy: 1.00000000
Batch number: 008, Training: Loss: 0.01458666, Accuracy: 1.00000000
Batch number: 009, Training: Loss: 0.01974454, Accuracy: 1.00000000
Batch number: 010, Training: Loss: 0.01269445, Accuracy: 1.00000000
Batch number: 011, Training: Loss: 0.

<h2> Create checkpoint for our Model </h2>

In [37]:

torch.save(model.state_dict(),'model.pt')
model.load_state_dict(torch.load('./model.pt'))
m=model.eval()
m

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

<H2>Validation<h2>

In [ ]:

history=[]
with torch.no_grad():
 
    # Set to evaluation mode
    m.eval()
 
    # Validation loop
    for j, (inputs, labels) in enumerate(validation_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
 
        # Forward pass - compute outputs on input data using the model
        outputs = m(inputs)
 
        # Compute loss
        loss = loss_func(outputs, labels)
 
        # Compute the total loss for the batch and add it to valid_loss
        valid_loss += loss.item() * inputs.size(0)
 
        # Calculate validation accuracy
        ret, predictions = torch.max(outputs.data, 1)
        correct_counts = predictions.eq(labels.data.view_as(predictions))
 
        # Convert correct_counts to float and then compute the mean
        acc = torch.mean(correct_counts.type(torch.FloatTensor))
 
        # Compute total accuracy in the whole batch and add to valid_acc
        valid_acc += acc.item() * inputs.size(0)
 
        print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
     
# Find average training loss and training accuracy
avg_train_loss = train_loss/len(train_indices)
avg_train_acc = train_acc/float(len(train_indices))
 
# Find average training loss and training accuracy
avg_valid_loss = valid_loss/len(val_indices)
avg_valid_acc = valid_acc/float(len(val_indices))
 
history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
         
epoch_end = time.time()
 
print("Epoch : {:03d}, Training: Loss: {:.8f}, Accuracy: {:.8f}%, \n\t\tValidation : Loss : {:.8f}, Accuracy: {:.8f}%, Time: {:.8f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))

<h2>Test<h2>

In [39]:

test_img=[]
for img in sub['image'] :
      img_path="./images/"+str(img)
      test_image = Image.open(img_path).convert('RGB')
      
      test_image_tensor = image_transform(test_image)
      if torch.cuda.is_available():
            test_image_tensor = test_image_tensor.view(1, 3, 224, 224).cuda()
      else:
            test_image_tensor = test_image_tensor.view(1, 3, 224, 224)

      #test_image_tensor = test_image_tensor.view(1, 3, 224, 224)
      with torch.no_grad():
      m.eval()
      out=m(test_image_tensor)
      ps = torch.exp(out)
      topk, topclass = ps.topk(1, dim=1,)
      sub.loc[sub['image']==img,'target']=float(ps[0][1])


In [40]:
sub.head()

,image,target
0,versewmzeguanbftslpjlyrupgqobt.jpg,0.000059
1,xxirvsjxpsdmicscgrmpcczbnrtoek.jpg,0.000032
2,pomnzkamhxjwtrgalmkbcevrqlijft.jpg,0.000271
3,imkumwldsvcwkjbhwglebnqtgjuaqy.jpg,0.999708
4,aelkivmayxgsdjosiibfgmkvfrjvjk.jpg,0.999848


<h2>Dispaly images<h2>

In [ ]:


show=sub
nrow = show.shape[0]
ncol = 1

fig = plt.figure(figsize=(4, 10)) 

gs = gridspec.GridSpec(nrow, ncol,
         wspace=0.0, hspace=0.0, top=0.95, bottom=0.05, left=0.17, right=0.845) 


for num, x in enumerate(show.image):
        img_path="./images/"+str(x)
        im= PIL.Image.open(img_path).convert('RGB')
        ax= plt.subplot(gs[num,0])
        fig = plt.figure()
        fig.set_size_inches(150,150)
        ax.imshow(im)
        ax.set_xticklabels([])
        ax.set_yticklabels([])

plt.show()

<h2>Create CSV File<h2>

In [42]:
# we can round prediction to improve score to 0.009 and , without them score=0.014
#sub.target=np.where(sub.target<0.18,0,sub.target)
#sub.target=np.where(sub.target>0.6,1,sub.target)
sub.to_csv('./FinalSub.csv',index=False)
sub

,image,target
0,versewmzeguanbftslpjlyrupgqobt.jpg,0.000059
1,xxirvsjxpsdmicscgrmpcczbnrtoek.jpg,0.000032
2,pomnzkamhxjwtrgalmkbcevrqlijft.jpg,0.000271
3,imkumwldsvcwkjbhwglebnqtgjuaqy.jpg,0.999708
4,aelkivmayxgsdjosiibfgmkvfrjvjk.jpg,0.999848
...,...,...
504,egmlopponygomfuzzhzoextlxzgkmo.jpg,0.000169
505,tmthqtrdrzwgadmsftvwpwlssemeic.jpg,0.000021
506,aljnaycepveqbedxhsklixyjodpcho.jpg,0.000261
507,ufjcnzoabfdevufathmxztkvmlfgmd.jpg,0.999969
